<a href="https://colab.research.google.com/github/TM0428/wsd2020/blob/main/system_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 必要パッケージのインストール

In [1]:
!pip install -U beautifulsoup4

Requirement already up-to-date: beautifulsoup4 in /usr/local/lib/python3.6/dist-packages (4.9.3)


In [4]:
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [3]:
!pip install gensim

In [6]:
# get html
import requests
import time
from pathlib import Path

def dirsName(name: str):
    return f"data/{name}/"

def fileName(name: str, id: str, extension: str="html")->str:
    return dirsName(name) + id.replace('/','_') + "." + extension #idの階層がローカルの階層にならないように

def urlName(article_url: str, article_id: str)->str:
    return article_url + article_id

def main():
    contents = [ # name, domain, article_urlは手動でせざるを得ないか、article_idを自動で取ってこられるようにしたい
        {
            "name": "sejuku",
            "domain":"https://www.sejuku.net/",
            "article_url":"https://www.sejuku.net/blog/",
            "article_id": ['33294', '3766', '105909'] # 記事idは https://www.sejuku.net/blog/curriculums-python から適当にとってきた。実際は https://www.sejuku.net/blog/archive/page/* から適当に取ってくるといいかもしれない
        },
        {
            "name": "techacademy",
            "domain":"https://techacademy.jp/",
            "article_url":"https://techacademy.jp/magazine/",
            "article_id": ['15508', '15636', '15571'] # 記事idは https://techacademy.jp/magazine/category/programming/python/page/44 
        },
        {
            "name": "note_nkmk",
            "domain":"https://note.nkmk.me/",
            "article_url":"https://note.nkmk.me/",
            "article_id": ['python-union-find', 'python-numpy-sin-con-tan', 'python-list-2d-sort'] # 記事idは https://note.nkmk.me/ の新着記事・更新記事から適当にとってきた
        },
        # https://docs.python.org/ はスクレイピングだめらしい
        
        #{
        #    "name": "python_docs",
        #    "domain":"https://docs.python.org/",
        #    "article_url":"https://docs.python.org/ja/3/library",
        #    "article_id": ['pathlib', 'random', 're'] # 記事idは https://docs.python.org/ja/3/library/ から適当にとってきた
        #},
        {
            "name": "qiita",
            "domain": "https://qiita.com/",
            "article_url": "https://qiita.com/", # https://qiita.com/をarticle_urlにするかhttps://qiita.com/user/をarticle_urlにするかは意見が分かれそう
            "article_id": ["kuroitu/items/f18acf87269f4267e8c1", "paleo_engineer/items/fdbde62df4394ddcaf11", # 記事idは https://qiita.com/kai_kou/items/180a91fd88dbbbd746f6 から適当に取ってきた
                            "UTOG/items/c77c75a6aa61ee93fc6d"] # idの階層がファイル名の階層になると階層構造が複雑になるので変換する必要がある
        }
        
    ]
    res = {}
    for content in contents:
        print(content)
        res[content['domain']] = []
        for id in content["article_id"]:
            filename = fileName(content["name"], id) # ローカル保存用のファイル名
            print(filename)
            file_path = Path(filename)
            print(file_path.exists())
            html_code:str
            if not file_path.exists(): # 1回getしたらローカルに保存して、複数回getしない
                urlname = urlName(content['article_url'], id) # get用のurl名
                print(urlname)
                html = requests.get(urlname) # getしたhtml(bytes)
                html_code = html.content.decode('utf-8') #getしたhtml(string)
                dir_path = Path(dirsName(content['name']))
                if not dir_path.exists():
                    dir_path.mkdir(mode=0o777,parents=True,exist_ok=False) # ディレクトリを作成
                with open(filename, mode='w', encoding='utf-8') as f:
                    f.write(html_code) # ローカルに保存
                time.sleep(3) # 適度なインターバル
            with open(filename, mode='r', encoding='utf-8') as f:
                html_code = f.read()
                res[content['domain']].append(html_code)
            
    return res




if __name__=="__main__":
    a = main()
    print(len(a['https://www.sejuku.net/'][0]))
    print(len(a['https://techacademy.jp/'][1]))
    print(len(a['https://note.nkmk.me/'][2]))
    print(len(a['https://qiita.com/'][0]))
    


{'name': 'sejuku', 'domain': 'https://www.sejuku.net/', 'article_url': 'https://www.sejuku.net/blog/', 'article_id': ['33294', '3766', '105909']}
data/sejuku/33294.html
True
data/sejuku/3766.html
True
data/sejuku/105909.html
True
{'name': 'techacademy', 'domain': 'https://techacademy.jp/', 'article_url': 'https://techacademy.jp/magazine/', 'article_id': ['15508', '15636', '15571']}
data/techacademy/15508.html
True
data/techacademy/15636.html
True
data/techacademy/15571.html
True
{'name': 'note_nkmk', 'domain': 'https://note.nkmk.me/', 'article_url': 'https://note.nkmk.me/', 'article_id': ['python-union-find', 'python-numpy-sin-con-tan', 'python-list-2d-sort']}
data/note_nkmk/python-union-find.html
True
data/note_nkmk/python-numpy-sin-con-tan.html
True
data/note_nkmk/python-list-2d-sort.html
True
{'name': 'qiita', 'domain': 'https://qiita.com/', 'article_url': 'https://qiita.com/', 'article_id': ['kuroitu/items/f18acf87269f4267e8c1', 'paleo_engineer/items/fdbde62df4394ddcaf11', 'UTOG/it

In [7]:
# html to text
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import re

# text_setを生成する
# text_setは各葉ノードのテキストと木構造を表す
def soup2textset(tag, text_set, tag_string):
    for content in tag.contents:
        if isinstance(content, NavigableString):
            #content.string = content.string.replace('\xa9', '(c)').replace('\xa0', '')
            text_set.add(tag_string + content.string)
        elif str.lower(content.name) in ["script", "meta", "style", "header", "footer"]:
            pass
        else:
            #print(content.name)
            soup2textset(content, text_set, tag_string + str.lower(content.name))
    return text_set

# text_setに一致する葉ノードを削除する
def removetags(tag, remove_text_set, tag_string):
    remove_lists = []
    for content in tag.contents:
        if isinstance(content, NavigableString):
            if (tag_string+content.string) in remove_text_set:
                remove_lists.append(content)
        elif str.lower(content.name) in ["script", "meta", "style", "header", "footer"]:
            remove_lists.append(content)
        else:
            removetags(content, remove_text_set, tag_string + str.lower(content.name))
    for content in remove_lists:
        content.extract()

from typing import List
# Webページを取得して解析する
# def html2text(htmls: list(str)) -> list(str):
# 同じドメインのhtml(string)群を受け取って、タグを削除したstringのlistを返す
def html2text(htmls: List[str]) -> List[str]:
    # 文字列抽出
    soups = [BeautifulSoup(html.replace("</li>", "\n</li>"), "html.parser") for html in htmls]
    #bodies = [soup.find("body").text for soup in soups]

    bodies = [soup.find("body") for soup in soups]

    # 重複の削除
    if 2 <= len(bodies):
        # 重複要素の列挙
        set0 = soup2textset(bodies[0], set(), "")
        set1 = soup2textset(bodies[1], set(), "")
        remove_set = frozenset(set0 & set1)
        if 3 <= len(bodies):
            set2 = soup2textset(bodies[2], set(), "")
            remove_set = frozenset(remove_set | (set2 & set0)  | (set2 & set1))
        #print(remove_set)
        # 重複要素の削除
        for i in range(len(bodies)):
            removetags(bodies[i], remove_set, "")

    #soup => textの変換
    for i in range(len(bodies)):
        bodies[i] = bodies[i].text

    # 空行削除
    emptyline_regex = re.compile('\n\\s*\n')
    for i in range(len(bodies)):
        bodies[i] = emptyline_regex.sub('\n', bodies[i])

    return bodies

def test():
    import os
    html_names = os.listdir("./data/sejuku")
    htmls = [open("./data/sejuku/" + html, 'r', encoding="utf-8").read() for html in html_names]
    bodies = html2text(htmls)

    print(html_names[0])
    print(bodies[0][:100])
    print("........")
    print(bodies[0][-100:])
    print("====================")
    print(html_names[1])
    print(bodies[1][:100])
    print("........")
    print(bodies[1][-100:])
    print("====================")
    print(html_names[2])
    print(bodies[2][:100])
    print("........")
    print(bodies[2][-100:])
    print("====================")
    # print(bodies[0].replace('\xa9', '(c)').replace('\xa0', ''))

if __name__=="__main__":
    test()



3766.html
Cloud9を導入すれば、簡単にPythonの開発環境を構築できます！Python学習したい！そう思ってもプログラミングを始めるところにすらたどり着けないという方もいらっしゃるかと思います。今回はCl
........
選してまとめてみた！第1回「プログラミング能力検定」12/7-13、受検料無料プログラミングを独学で学習する方法と挫折をなくす3つのコツ初心者向けプログラミング言語おすすめランキング【2020年最新】
33294.html

Pythonを自分のPCにインストールしたいけどどうやったらいいの？
Pythonのバージョンを確認するには？と疑問を持たれている方もいるのではないでしょうか？言語を本格的に学ぶためには環境を構築す
........
（ただし暫く何も足さない）がいいんじゃないかというメモ - min.t (ミント)未経験からフリーランスエンジニアとして仕事獲得するまでの全手順プログラミング初心者は何からすべき？習得までの7ステップ
105909.html

Pythonの動かし方がいまいちわからない……
実行したらエラーが発生したけど、どうすればいいの？
おすすめの実行環境があったら教えてほしいこれからPythonを学習しようとしている初心者の方にとっ
........
く何も足さない）がいいんじゃないかというメモ - min.t (ミント)
【2020年度の合格者速報】名門私立小学校受験・幼稚園受験なら「AIペーパー分析」×「マンツーマン指導」のアイキューキッズ
 


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import MeCab
from gensim.models.doc2vec import Doc2Vec
import os


class Embedder:
    def __init__(self, model_path="/content/drive/MyDrive/東工大/システム設計演習/jawiki.doc2vec.dbow300d/jawiki.doc2vec.dbow300d.model"):
        """
        constructor

        Args:
            model_path (str, optional): Defaults to "jawiki.doc2vec.dbow300d.model".
        """
        self.model = Doc2Vec.load(model_path)

    def _tokenize(self, text):
        """
        分かち書きを行うメソッド

        Args:
            text (str): 入力文字列

        Returns:
            list[str]: 分かち書き済み単語リスト
        """
        wakati = MeCab.Tagger("-Owakati")
        wakati.parse("")
        return wakati.parse(text).strip().split()

    def doc2vec(self, text):
        """
        doc2vec

        Args:
            text (str): string

        Returns:
            numpy.array: embedded vector
        """
        return self.model.infer_vector(self._tokenize(text))


def main():
    text = 'はじめに今まで自分が使ってきた中で、これは生産性が爆上げする！と思うものを厳選しました是非最後までご覧ください'
    embedder = Embedder()
    print(embedder.doc2vec(text))


if __name__ == "__main__":
  # データ一覧
  html_names = os.listdir("./data/sejuku")
  htmls = [open("./data/sejuku/" + html, 'r', encoding="utf-8").read() for html in html_names]
  bodies = html2text(htmls)
  embedder = Embedder()
  for body in bodies:
    print(embedder.doc2vec(body))

[-0.04644709 -0.08066528  0.00238143  0.18936065  0.05107804 -0.1655564
  0.07017115 -0.03674507 -0.25644907 -0.05919756  0.19240296 -0.29809216
  0.05338582 -0.1413264   0.52438015  0.09273072  0.29368812  0.00487726
  0.18998733 -0.0972043  -0.00554163  0.11979109 -0.19052498  0.01486254
 -0.1786719   0.09614064 -0.08725657  0.00515512 -0.09155009  0.14219287
 -0.09460672  0.08398268  0.02884137  0.08827341  0.08961735 -0.21750721
 -0.3219371  -0.13661276  0.14393029 -0.17469698  0.02844832  0.1189246
 -0.07063111 -0.16772404  0.05942083  0.24469067  0.14092228 -0.20148261
  0.17495942  0.13248889  0.18743983  0.01487395 -0.0741148  -0.31480026
  0.10368635 -0.14284661 -0.39721352 -0.0464507   0.2101486  -0.05343342
  0.5460407   0.16322234 -0.33409584  0.07207932 -0.13186441 -0.24903716
 -0.02862005 -0.08145452  0.15269168  0.11667015  0.00494997 -0.13526516
  0.15241852  0.14982983 -0.04387985  0.12656371 -0.02558896 -0.12555744
  0.37873355 -0.02303609  0.226243   -0.17561366  0.1